# Supervised/unsupervised Sentiment and Topic analysis. <br>

### Final project assignment. <br>

Introduction  to Natural Language Processing (NLP) 
(DSCI-D590-31731)

Anitha Ganapathy | aganapa@iu.edu <br>
12/01/2020

In [1]:
!nvidia-smi

Sun Jan 17 00:17:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.81       Driver Version: 456.81       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX150      WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   50C    P8    N/A /  N/A |     64MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !nvcc --version

In [3]:
# !pip install docx2txt
# !pip install tensorflow_text


Organizing imports.

In [4]:
import nltk
import os
import sys
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import docx2txt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import seaborn as sns
import gensim
import json
import datetime

from tqdm import tqdm
from nltk import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import wordnet
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing import text, sequence
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE
from gensim.models import word2vec
from pylab import rcParams
from pandas.plotting import register_matplotlib_converters

# plt.style.use('ggplot')
%config InlineBackend.figure_format='retina'
%matplotlib inline

import warnings 
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'nltk'

In [ ]:
nltk.download('stopwords')

In [ ]:
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 2000737430
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

In [ ]:
print(os.listdir("../content"))
print(f'Python version     : {sys.version}')
print(f'Pandas version     : {pd.__version__}')
print(f'Numpy version      : {np.__version__}')
print(f'Tensorflow version : {tf.__version__}')
print(f'NLTK version       : {nltk.__version__}')
print(f'Regex version      : {re.__version__}')
print("GPU Device name: ",tf.config.list_physical_devices('GPU'))
# print("\nCheck if GPU is available: ", tf.test.is_gpu_available())

**Dataset**
The data I am using for the assignment is the Amazon Fine Food Reviews. <br>
https://www.kaggle.com/snap/amazon-fine-food-reviews
 

**Data Reference:**

J. McAuley and J. Leskovec. From amateurs to connoisseurs: modeling the evolution of user expertise through online reviews. WWW, 2013.

In [ ]:
df = pd.read_csv("Amazon Fine Food Reviews.csv", parse_dates= True)

## Step 1 :  Describe data.

In [ ]:
print("The Amazon Fine Food Reviews dataset brief:")
print("\nNumber of reviews:", df.shape[0])
print("Number of users: ", len(df.UserId.unique()))
print("Number of products: ", len(df.ProductId.unique()) )
print("Number of Attributes/Columns in data: ", len(df.columns))

In [ ]:
# df.head()

The column or features in the dataset: <br>
*   Id <br>
*   ProductId — unique identifier for the product <br>
*   UserId — unqiue identifier for the user
*   ProfileName <br>
*   HelpfulnessNumerator — number of users who found the review helpful <br>
*   HelpfulnessDenominator — number of users who indicated whether they found the review helpful or not <br>
*   Score — rating between 1 and 5 <br>
*   Time — timestamp for the review <br>
*   Summary — brief summary of the review <br>
*   Text — text of the review <br>







In [ ]:
df.info()

In [ ]:
df.describe()

Checking for NAN values in the dataset. Nan values in the text column provided difficulty in plotting the wordcloud.
So replaced the nan with empty string.


In [ ]:
df.isna().sum()             

## Step 2: Perform data preprocessing.

In [ ]:
#df = pd.read_csv("Amazon Fine Food Reviews.csv", parse_dates= True)

df_old = df
df.Summary = df.Summary.fillna('')       # replacing the NAN with ''
df['review'] = df['Summary'] + df['Text']
df.isna().sum()    

In [ ]:
df = df[['Score', 'review']]
df.head()

In [ ]:
df.Score.value_counts()

In [ ]:
df.isna().sum()    

### Analysing the data. 

In [ ]:
# sns.countplot(df['Score'], order=df.Score.value_counts().index)
sns.countplot(df['Score'])
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Amazon Food and Wine Reviews.')

Just for simplicity and because we need more data for the binary classification of Good or Bad reviews, we shall consider a rating of 3 or less as bad reviews and a rating of 4 and more as good reviews

In [ ]:
# Give reviews with Score > 3 a positive rating('good), 
# and reviews with a score<=3 a negative review_type('bad').

df["review_type"] = df["Score"].apply(lambda x: "negative" if x <= 3 else "positive")

In [ ]:
df.review_type.value_counts()

In [ ]:
sns.countplot(
  x='review_type',
  data=df,
  order=df.review_type.value_counts().index
)
plt.xlabel("type")
plt.title("Review type")

In [ ]:
positive_reviews = df[df.review_type == "positive"]
negative_reviews = df[df.review_type == "negative"]

print("Good reviews shape: ",positive_reviews.shape)
print("Bad reviews shape: ", negative_reviews.shape)

We need the good and bad reviews to have same review counts, so we will take equal amout of good reviews as bad.

In [ ]:
positive_reviews_text = " ".join(positive_reviews.review.to_numpy().tolist())
negative_reviews_text = " ".join(negative_reviews.review.to_numpy().tolist())


In [ ]:
from nltk.corpus import stopwords
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
def normalize_document(doc):
    # lowercase and remove special characters\whitespace
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, flags =re.I)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc
  


In [ ]:
positive_normalize_text = normalize_document(positive_reviews_text)
negative_normalize_text = normalize_document(negative_reviews_text)


In [ ]:
datetime.datetime.now()
positive_reviews_cloud = WordCloud(stopwords=STOPWORDS,background_color="black").generate(positive_normalize_text)
negative_reviews_cloud = WordCloud(stopwords=STOPWORDS, background_color="black").generate(negative_normalize_text)
datetime.datetime.now()

In [ ]:
print(datetime.datetime.now())
def show_word_cloud(cloud, title):
  plt.figure(figsize = (20, 20))
  plt.imshow(cloud, interpolation='bilinear')
  plt.title(title)
  plt.axis("off")
  plt.show()
print(datetime.datetime.now())

## Step 3: Topic Analysis

In [ ]:
show_word_cloud(positive_reviews_cloud, "Positive Reviews cloud display.")

In [ ]:
show_word_cloud(negative_reviews_cloud, "Negative Reviews cloud display.")

We’ll deal with the review type count imbalance by sampling the number of good ones to that of the bad ones. We need same amount of positive and negative reviews.

In [ ]:
positive_df = positive_reviews.sample(n=len(negative_reviews), random_state=RANDOM_SEED)
negative_df = negative_reviews

In [ ]:
review_df = positive_df.append(negative_df).reset_index(drop=True)
print("Final review data df shape: ", review_df.shape)

In [ ]:
sns.countplot(
  x='review_type',
  data=review_df,
  order=review_df.review_type.value_counts().index
)

plt.xlabel("type")
plt.title("Review type (resampled)")

google/universal-sentence-encoder/4 - a much larger model yielding 512 dimensional embeddings trained with a deep averaging network (DAN) encoder.

The Universal Sentence Encoder encodes text into high dimensional vectors that can be used for text classification, semantic similarity, clustering, and other natural language tasks.



In [ ]:
import tensorflow_hub as hub

# it took 14 mins to load this module

print(datetime.datetime.now())
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
print(datetime.datetime.now())

In [ ]:
sent_1 = ["the location is great"]
sent_2 = ["amazing location"]

emb_1 = use(sent_1)
emb_2 = use(sent_2)
emb_1.shape

In [ ]:
np.inner(emb_1, emb_2).flatten()[0]

## Step 4. Splitting data into training and testing.
Split in 80-20 training or UseCross-validation

In [ ]:
type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  review_df.review_type.to_numpy().reshape(-1, 1)
)

In [ ]:
train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    review_df.review, 
    type_one_hot, 
    test_size=.2, 
    random_state=RANDOM_SEED
  )

## Step 5: Perform data vectorization.

In [ ]:
print(datetime.datetime.now())
X_train = []
for r in tqdm(train_reviews):
  embedded = use(r)
  review_emb = tf.reshape(embedded, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)
print(datetime.datetime.now())

In [ ]:
print(datetime.datetime.now())
X_test = []
for r in tqdm(test_reviews):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(review_emb)

X_test = np.array(X_test)
print(datetime.datetime.now())

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
print(y_train.shape, y_test.shape)

## Step 6: Sentiment analysis

Sentiment Analysis is a binary classification problem. Let’s use Keras to build a model.

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Dense(units=256, input_shape=(X_train.shape[1], ), activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

The model is composed of 2 fully-connected hidden layers. Dropout is used for regularization.

We’ll train for 15 epochs and use 10% of the data for validation:

In [ ]:
print(datetime.datetime.now())

history = model.fit(
    X_train, y_train, 
    epochs=15, 
    batch_size=16, 
    validation_split=0.1, 
    verbose=1, 
    shuffle=True
)

print(datetime.datetime.now())

In [ ]:
rcParams['figure.figsize'] = 12, 8
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel("epoch")
plt.ylabel("Cross-entropy loss")
plt.legend();

In [ ]:
rcParams['figure.figsize'] = 12, 8
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend();

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
print("Testing Accuracy:  {:.4f}".format(accuracy))

## Step 7: Predicting Sentiment

Example : 1

In [ ]:
print(test_reviews.iloc[0])
print("Sentiment Analysis is :")
print("Bad" if y_test[0][0] == 1 else "Good")

In [ ]:
y_pred = model.predict(X_test[:1])
print(y_pred)
"Bad" if np.argmax(y_pred) == 0 else "Good"

Example 2:

In [ ]:
print(test_reviews.iloc[1])
print("Bad" if y_test[1][0] == 1 else "Good")


In [ ]:
y_pred = model.predict(X_test[1:2])
print(y_pred)
"Bad" if np.argmax(y_pred) == 0 else "Good"

Example 3:

In [ ]:
print(test_reviews.iloc[5])
print("Bad" if y_test[5][0] == 1 else "Good")

In [ ]:
y_pred = model.predict(X_test[5:6])
print(y_pred)
"Bad" if np.argmax(y_pred) == 0 else "Good"

## Step 8: Confusion matrix

In [ ]:
# confusion matrix in sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras import backend as K
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

labels=['positive','negative']

#Confusion Matrix and Classification Report
#confusion matrix
y_pred=model.predict(X_test)
#y_pred = (y_pred > 0.5)

#cm =confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
#print("Confusion Matrix : \n", cm)

In [ ]:
confusion_df = pd.DataFrame(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)),
             columns=["Predicted Class " + str(class_name) for class_name in [0,1]],
             index = ["Class " + str(class_name) for class_name in [0,1]])

print(confusion_df)

In [ ]:
f, ax=plt.subplots(figsize=(5,5))
sns.heatmap(cm,annot=True,linewidths=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

## Step 9: Classification Report.

Precision Score <br>
TP – True Positives <br>
FP – False Positives<br>

Precision – Accuracy of positive predictions. <br>
Precision = TP/(TP + FP)<br>

In [ ]:
from sklearn.metrics import precision_score

print("Precision score: {}".format(precision_score(y_test.argmax(axis=1),y_pred.argmax(axis=1))))

Recall Score <br>
FN – False Negatives <br>

Recall (aka sensitivity or true positive rate): <br>
Fraction of positives That were correctly identified. <br>
Recall = TP/(TP+FN) <br> 



In [ ]:
from sklearn.metrics import recall_score

print("Recall score: {}".format(recall_score(y_test.argmax(axis=1),y_pred.argmax(axis=1))))

F1 Score <br>
F1 Score (aka F-Score or F-Measure) – A helpful metric for comparing two classifiers. F1 Score takes into account precision and the recall. It is created by finding the the harmonic mean of precision and recall.
<br>
F1 = 2 x (precision x recall)/(precision + recall)

In [ ]:
from sklearn.metrics import f1_score

print("F1 Score: {}".format(f1_score(y_test.argmax(axis=1),y_pred.argmax(axis=1))))

#### Classification Report
Report which includes Precision, Recall and F1-Score.

In [ ]:
print("CLASSIFICATION REPORT: \n")
cr = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(cr)

## Step 10: Conclusion


1.   We started with analyzing and describing the data and providing a few metrics.
2.   We then proceeded with data preprocessing and creating the word clouds for the positive and negative reviews.
3.   We used the USE for word embeddings and tested our data with the Keras model.
4.   We achieved an accuracy of 91% and 90% for the negative and positive reviews respectively.
5.   The full confusion matrix and the classification reports are detailed as above.
6.  We also checked few test data as well.





## Step 11: Reference:

https://curiousily.com/posts/sentiment-analysis-with-tensorflow-2-and-keras-using-python/

https://realpython.com/python-keras-text-classification/


https://joshlawman.com/metrics-classification-report-breakdown-precision-recall-f1/

# Step 12: The END